## Creacion de Funciones para API

Vamos a desarrollar y evaluar las funciones clave de nuestra API utilizando los datos disponibles. Este proceso nos permite confirmar la efectividad y eficiencia de las funciones en un entorno controlado. Sin embargo, al implementarlas en FastAPI y Render, es posible que necesitemos ajustes adicionales para cumplir con sus requisitos específicos

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Funcion UserForGenre

In [3]:
df_UserForGenre = pd.read_csv("data_repo\df_UsersForGenre.csv")
df_UserForGenre

,user_id,main_genre,year_posted,playtime_hour
0,--000--,Action,2014,49.150000
1,--ace--,Action,2014,21.150000
2,--ionex--,Action,2015,14.033333
3,-Azsael-,Indie,2013,198.150000
4,-Beave-,Action,2014,47.733333
...,...,...,...,...
19124,zvanik,Action,2015,382.233333
19125,zynxgameth,Action,2013,64.100000
19126,zyr0n1c,Action,2013,40.966667
19127,zyr0n1c,Action,2014,10.716667


In [11]:
def UserForGenre( genero : str ):
    """
    Ingrese genero y devuelve el usuario on más horas jugadas  
    y una lista de la acumulación de horas jugadas por año.
    """
    genero_1 = df_UserForGenre[df_UserForGenre["main_genre"]== genero]
    user_max = genero_1.loc[genero_1["playtime_hour"].idxmax()]["user_id"]
    horas_x_año = genero_1.groupby(["year_posted"])["playtime_hour"].sum().reset_index()
    horas_lista = horas_x_año.rename(columns={'year_posted': 'Año', 'playtime_hour': 'Horas'}).to_dict(orient="records")
    result_ug = {
        f"Usuario con más horas jugadas para Género: {genero}": user_max,
        "Horas jugadas": horas_lista
    }
    return result_ug

In [85]:
UserForGenre("Adventure")

{'Usuario con más horas jugadas para Género: Adventure': '76561197972452208',
 'Horas jugadas': [{'Año': 2010, 'Horas': 5.233333333333333},
  {'Año': 2011, 'Horas': 16.5},
  {'Año': 2012, 'Horas': 463.45},
  {'Año': 2013, 'Horas': 18430.966666666667},
  {'Año': 2014, 'Horas': 8068.583333333333},
  {'Año': 2015, 'Horas': 24429.566666666666}]}

## Funcion BestDeveloper

In [40]:
df_BestDeveloper = pd.read_csv("data_repo\df_BestDeveloper.csv")
df_BestDeveloper

,year_posted,item_id,developer,recommendation_count
0,2011,440,Valve,1
1,2011,18700,Broken Rules,1
2,2011,33460,Ubisoft Montpellier,1
3,2012,440,Valve,1
4,2012,42920,NeoCoreGames,1
5,2012,55110,Volition,1
6,2013,275850,Hello Games,52
7,2013,730,Valve,32
8,2013,440,Valve,25
9,2014,221100,Bohemia Interactive,19


In [140]:
def best_developer_year(año: int):
    """
    Ingrese año y devuelve el top 3 de desarrolladores con juegos MAS 
    recomendados por usuarios.
    """
    if type(año) != int:
        return {"Debes colocar el año en entero, Ejemplo:2012"}
    if año < df_BestDeveloper["year_posted"].min() or año > df_BestDeveloper["year_posted"].max():
        return {"Año no encontrado "}
    df_año = df_BestDeveloper[df_BestDeveloper["year_posted"] == año]
    df_ordenado_recomendacion = df_año.sort_values(by="recommendation_count", ascending=False)
    top_3_developers = df_ordenado_recomendacion.head(3)[["developer", "recommendation_count"]]
    
    # Crear el resultado en el formato deseado
    result_bd = [{"Puesto {}: ".format(i+1) + top_3_developers.iloc[i]['developer'] for i in range(len(top_3_developers))}]
    
    return result_bd



In [141]:
best_developer_year(2014)


[{'Puesto 1: Bohemia Interactive',
  'Puesto 2: Valve',
  'Puesto 3: Trek Industries, Inc'}]

## Funcion DeveloperReviews

In [45]:
df_DeveloperReviews = pd.read_csv("data_repo/df_DeveloperReviews.csv")
df_DeveloperReviews

,developer,sentiment_analysis,sentiment_analysis_count
0,07th Expansion,0,1
1,07th Expansion,1,2
2,"10th Art Studio,Adventure Productions",0,1
3,"10th Art Studio,Adventure Productions",1,1
4,10tons Ltd,1,1
...,...,...,...
3977,"インレ,Inre",1,4
3978,橘子班,0,1
3979,橘子班,1,2
3980,橘子班,2,1


In [142]:
def developer_reviews_analysis(desarrolladora: str):
    """
    Ingrese desarrolladora y devuelve un diccionario con el nombre de la desarrolladora como llave y una lista 
    con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con 
    un análisis de sentimiento como valor.
    """
    if type(desarrolladora) != str:
        return "Debes colocar un developer de tipo str, EJ:'07th Expansion'"
    if len(desarrolladora) == 0:
        return "Debes colocar un developer en tipo String"
    
    # Se filtra el DataFrame por la desarrolladora dada
    df_developer = df_DeveloperReviews[df_DeveloperReviews["developer"] == desarrolladora]
    
    # Se suma las cuentas de análisis de sentimiento por categoría
    sentiment_counts = df_developer.groupby("sentiment_analysis")["sentiment_analysis_count"].sum().to_dict()
    
    # Se mapea las categorías de análisis de sentimiento a nombres más descriptivos, excluyendo "Neutral"
    sentiment_dicc = {0: "Negativo", 2: "Positivo"}
    
    # Se crea el diccionario resultante
    result_dr = {desarrolladora: [f"{sentiment_dicc[key]} = {value}" for key, value in sentiment_counts.items() if key in sentiment_dicc]}
    
    return result_dr

In [143]:
developer_reviews_analysis('Valve')

{'Valve': ['Negativo = 1058', 'Positivo = 4330']}

## Funcion Developer

In [23]:
df_Developer = pd.read_csv("data_repo/df_Developer.csv")
df_Developer

,release_year,developer,cantidad_id,contenido free
0,1983,Digital Leisure Inc.,1,0.0
1,1984,Digital Leisure Inc.,1,0.0
2,1984,"Ed Hobbs,Robert Crossfield",1,0.0
3,1985,"MicroProse Software, Inc",1,0.0
4,1986,Choose Multiple LLC,1,0.0
...,...,...,...,...
15019,2018,杭州分浪网络科技有限公司,1,100.0
15020,2019,Ninetales Studios,1,0.0
15021,2019,Nodding Heads Games,1,100.0
15022,2019,Poetic Justice Studios,1,100.0


In [24]:
def developer(desarrollador: str):
    """
    Ingrese desarrollador y devuelve un Diccionario con años como clave y Cantidad de Items y Porcentaje de Contenido Free
    """
    if type(desarrollador) != str:
        return {"Error": "Debes colocar un desarrollador de tipo str, por ejemplo: 'Valve'"}
    if len(desarrollador) == 0:
        return {"Error": "Debes colocar un desarrollador en tipo String"}
    
    df_desarrollador = df_Developer[df_Developer["developer"] == desarrollador]
    df_desarrollador_subset = df_desarrollador[["release_year", "cantidad_id", "contenido free"]]
    df_desarrollador_subset["contenido_free_porcentaje"] = df_desarrollador_subset["contenido free"].round(2)
    result_dict = {}
    for year, data in df_desarrollador_subset.groupby("release_year"):
        result_dict[year] = {str(year): {'Cantidad de Items': cantidad, 'Contenido Free': porcentaje} for cantidad, porcentaje in data[["cantidad_id", "contenido_free_porcentaje"]].values}
    reformatted_result = {year: values[str(year)] for year, values in result_dict.items()}
    return reformatted_result


In [26]:
developer("Valve")


{1998: {'Cantidad de Items': 1.0, 'Contenido Free': 0.0},
 1999: {'Cantidad de Items': 1.0, 'Contenido Free': 0.0},
 2000: {'Cantidad de Items': 2.0, 'Contenido Free': 0.0},
 2001: {'Cantidad de Items': 1.0, 'Contenido Free': 0.0},
 2003: {'Cantidad de Items': 1.0, 'Contenido Free': 0.0},
 2004: {'Cantidad de Items': 5.0, 'Contenido Free': 0.0},
 2005: {'Cantidad de Items': 1.0, 'Contenido Free': 100.0},
 2006: {'Cantidad de Items': 2.0, 'Contenido Free': 0.0},
 2007: {'Cantidad de Items': 3.0, 'Contenido Free': 33.33},
 2008: {'Cantidad de Items': 1.0, 'Contenido Free': 0.0},
 2009: {'Cantidad de Items': 1.0, 'Contenido Free': 0.0},
 2010: {'Cantidad de Items': 2.0, 'Contenido Free': 50.0},
 2011: {'Cantidad de Items': 1.0, 'Contenido Free': 0.0},
 2012: {'Cantidad de Items': 2.0, 'Contenido Free': 50.0},
 2014: {'Cantidad de Items': 7.0, 'Contenido Free': 100.0},
 2016: {'Cantidad de Items': 1.0, 'Contenido Free': 100.0},
 2017: {'Cantidad de Items': 2.0, 'Contenido Free': 0.0}}

## Funcion UserData

In [3]:
df_UserData = pd.read_csv("data_repo/df_UserData.csv")
df_UserData

,user_id,items_count,price_numeric,recommend
0,--000--,58,146.88,1
1,--ace--,44,115.87,2
2,--ionex--,23,59.96,2
3,-2SV-vuLB-Kg,68,232.66,5
4,-Azsael-,167,1104.41,1
...,...,...,...,...
22236,zwanzigdrei,93,431.68,1
22237,zy0705,6,0.00,1
22238,zynxgameth,50,279.75,1
22239,zyr0n1c,73,352.75,9


In [8]:
def userdata(user_id: str):
    """
    Ingrese id de usuario y devuelve la cantidad de dinero gastado, el porcentaje de 
    recomendación y cantidad de juegos.
    """
    user_data = df_UserData[df_UserData['user_id'] == user_id]
    
    if user_data.empty:
        return {"Error": f"No se encontraron datos para el user_id '{user_id}'"}
    
    price_numeric_total = user_data.iloc[0]['price_numeric']
    recommend_percentage = (user_data['recommend'].sum() / len(user_data)) * 100
    items_count = user_data.iloc[0]['items_count']
    result = {
        "user_id": user_id,
        "price_numeric": f"{price_numeric_total} USD",
        "% de recomendación": f"{recommend_percentage}%",
        "items_count": items_count
    }
    
    return result

In [9]:
userdata('zynxgameth')

{'user_id': 'zynxgameth',
 'price_numeric': '279.75 USD',
 '% de recomendación': '100.0%',
 'items_count': 50}